In [40]:
import os
from dotenv import load_dotenv

# hide sensible information
load_dotenv(r"snowflake.env")

#this is the library needed to establish a snowflake connection with python (and anaconda in my case)
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd

In [41]:
# let's import my honeyproduction CSV-table into a pandas dataframe

df= pd.read_csv('honeyproduction.csv')
df.columns= [x.upper()  for x in df.columns]
df.head()

,STATE,NUMCOL,YIELDPERCOL,TOTALPROD,STOCKS,PRICEPERLB,PRODVALUE,YEAR
0,AL,16000.0,71,1136000.0,159000.0,0.72,818000.0,1998
1,AZ,55000.0,60,3300000.0,1485000.0,0.64,2112000.0,1998
2,AR,53000.0,65,3445000.0,1688000.0,0.59,2033000.0,1998
3,CA,450000.0,83,37350000.0,12326000.0,0.62,23157000.0,1998
4,CO,27000.0,72,1944000.0,1594000.0,0.70,1361000.0,1998


In [42]:
# save all env-variables into notebook-variables
SF_USER = os.environ.get("SF_USER")
SF_PASSWORD = os.environ.get("SF_PASSWORD")
SF_ACCOUNT_ID = os.environ.get("SF_ACCOUNT_ID")
DATABASE_NAME = os.environ.get("DATABASE_NAME")
SCHEMA_NAME = os.environ.get("SCHEMA_NAME")

In [44]:
# Connect to snowflake with your user-specific information

cnx = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT_ID,
    database=DATABASE_NAME,
    schema=SCHEMA_NAME,
    warehouse="TEST_WAREHOUSE",
    role='ACCOUNTADMIN'
)
cs= cnx.cursor()

In [45]:
# create a table with these columns and datatypes with the following command 

create_table = 'CREATE OR REPLACE TABLE MARCH.CDM.SOMETHING (column1 VARCHAR, column2 VARCHAR, column3 VARCHAR)'
cs.execute(create_table)

In [34]:
# write the table with the following command
write_pandas(cnx, df, table_name="HONEY")

(True,
 1,
 626,
 [('irbhinlmiu/file0.txt', 'LOADED', 626, 626, 1, 0, None, None, None, None)])

In [37]:
# execute SQL queries this way and print them if you want to

cs.execute("SELECT STATE, TOTALPROD, YEAR FROM MARCH.CDM.HONEY LIMIT 10;")
rows= cs.fetchall()
for row in rows:
    print(row)

('AL', '1136000', 1998)
('AZ', '3300000', 1998)
('AR', '3445000', 1998)
('CA', '37350000', 1998)
('CO', '1944000', 1998)
('FL', '22540000', 1998)
('GA', '4200000', 1998)
('HI', '944000', 1998)
('ID', '6e+06', 1998)
('IL', '639000', 1998)


In [38]:
# load a snowflake table into a pandas dataframe with this method

cs.execute("SELECT * FROM CAR_DATA")

df_from_snowflake = cs.fetch_pandas_all()
df_from_snowflake

,CAR_NAME,YEAR,SELLING_PRICE,PRESENT_PRICE,KMS_DRIVEN,FUEL_TYPE,SELLER_TYPE,TRANSMISSION,OWNER
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0
